In [1]:
using Revise
using KLIEPInference
using Distributions
using LinearAlgebra

In [2]:
θ = zeros(5, 5)
spl = IsingSampler(θ)
X = rand(spl, 100)
Y = rand(spl, 500)
Ψx = Ψising(X)
Ψy = Ψising(Y);

In [6]:
#θhat = KLIEP(Ψx, Ψy,  Mosek_KLIEP())
@time θhat = KLIEP(Ψx, Ψy,  SCS_KLIEP())

  1.298192 seconds (114.35 k allocations: 11.015 MiB, 0.62% gc time)
----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 6500, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 511, constraints m = 1501
Cones:	linear vars: 1
	exp vars: 1500, dual exp vars: 0
Setup time: 3.27e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.10e+00  5.57e-01  6.53e-02 -1.39e+01 -1.21e+01  2.99e-15  7.76e-03 
   100| 2.33e-02  1.23e-02  8.13e-02 -4.35e-02 -1.40e-01  6.68e-02  3.

10-element Array{Float64,1}:
  0.11464510407379162 
  0.09657227807667636 
  0.1521473733277225  
 -0.18495905960619297 
  0.058401335332394236
  7.113976949482982e-5
 -0.1713434558461733  
 -0.09981989444407603 
  0.12210947002108315 
  0.08754762903255835 

In [5]:
@time θhat1 = KLIEP(Ψx, Ψy,  CD_KLIEP())
norm(θhat1 - θhat, Inf)

  0.003361 seconds (504 allocations: 1.910 MiB)


9.759016308355517e-6

In [ ]:
θx = zeros(5, 5)
splx = IsingSampler(θx)
X = rand(splx, 50)

θy = zeros(5, 5)
θy[1, 2] = .3
θy[2, 1] = .3
sply = IsingSampler(θy)
Y = rand(sply, 100)

Ψx = Ψising(X)
Ψy = Ψising(Y);

In [ ]:
θhat = KLIEP(Ψx, Ψy,  SCS_KLIEP())

In [ ]:
@time θhat = spKLIEP(Ψx, Ψy,  0.1, SCS_KLIEP())

In [ ]:
@time θhat1 = spKLIEP(Ψx, Ψy,  0.1, CD_KLIEP())

In [ ]:
norm(θhat1 - θhat, Inf)